In [21]:
#Here we import necessary packages 
import pandas as pd
import nltk
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
import functions
import twitter
import pickle

In [20]:
#static variable saves how many times the program has been run 
run_var = 0

In [65]:
text_data = pd.read_csv('tweets.csv')
df = pd.DataFrame({"Tweet": text_data['text'][0:20], "Status": text_data['category'][0:20]})

In [55]:
#df = df.sort_values(by=['Time'])
df = df.dropna(axis=0, how='all')
df = df.reset_index()

In [56]:
f = pd.DataFrame({"Tweet": df['Tweet'], "Status": text_data['category']})
print(f)
s = pd.DataFrame({"Tweet": df['Tweet'], "Status": text_data['category']}).truncate(after = 199999)

       Status                                              Tweet
0         bot    #ThingsDoneByMistake kissing auntie in the lips
1         bot  RT @mc_derpin: #TheOlderWeGet the more pessimi...
2         bot  RT @dmataconis: Ready To Feel Like A Failure? ...
3         bot    Amen! #blacklivesmatter https://t.co/wGffaOqgzl
4         bot  RT @NahBabyNah: Twitchy: Chuck Todd caught out...
5         bot  RT @mcicero10: #BernieSanders #Trump people sh...
6         bot  RT @ItsJustJaynie: @HillaryClinton The undecid...
7         bot                           @TodayCleveland 'no way'
8         bot  @NickTomaWBRE Hi, Nick! We're holding a "Miner...
9         bot  What. Is. A. Resolution #My4WordNewYearsResolu...
10        bot  Lifetime movie   your pet   psycho neighbor = ...
11        bot  RT @Conservatexian: New post: "UN alarm that m...
12        bot  RT @HillaryClinton: This one's for you, Hillar...
13        bot  RT @leonpui_: Hillary Clinton, Obama and the D...
14        bot  5th grade.

In [47]:
w = s['Tweet'].str.lower()

In [42]:
df['Tweet'].to_csv(r'russian.txt', header=None, index=None, sep=' ', mode='a')
content = open('russian.txt', encoding='utf8').read()
words = nltk.word_tokenize(content)

In [64]:
# 2016 Nominees
trump = 'realDonaldTrump'
clinton = 'HillaryClinton'

# Top 6 Prospects
biden = 'JoeBiden'
booker = 'CoryBooker'
sanders = 'BernieSanders'

harris = 'KamalaHarris'
gillibrand = 'SenGillibrand'
warren = 'elizabethforma'

# DataFrames for favorites/retweets/tweets for each politician
trump_data = twitter.tweets_favorites_retweets(trump)
clinton_data = twitter.tweets_favorites_retweets(clinton)

biden_data = twitter.tweets_favorites_retweets(biden)
booker_data = twitter.tweets_favorites_retweets(booker)
sanders_data = twitter.tweets_favorites_retweets(sanders)

harris_data = twitter.tweets_favorites_retweets(harris)
gillibrand_data = twitter.tweets_favorites_retweets(gillibrand)
warren_data = twitter.tweets_favorites_retweets(warren)

tweets = trump_data['tweets']
tweets = tweets.append(clinton_data['tweets'])
tweets = tweets.append(biden_data['tweets'])
tweets = tweets.append(booker_data['tweets'])
tweets = tweets.append(sanders_data['tweets'])
tweets = tweets.append(harris_data['tweets'])
tweets = tweets.append(gillibrand_data['tweets'])
tweets = tweets.append(warren_data['tweets'])

traits = ["human"] * tweets.size
out = pd.DataFrame({"Tweet": df['Tweet'], "Status": text_data['category']})

In [62]:
#We need to first create a list of feature dictionaries from the tokenized tweets will populate
#Entire tweets are tokenized into lists of invidiual words, of which each element is associated with value pairs of "bot" or "human" 
#to label the category the data belongs to 

fake_tokens = []
real_tokens = []

#tokenize each tweet then add to fake_tokens (list with confirmed bot tweet content)
for i in range(0, s.size):
    item = nltk.word_tokenize(s[i]["Tweet"])
    fake_tokens.append(item)
    
#tokenize each tweet then add to real_tokens (list with confirmed human tweet content)    
for i in range(0, out.size):
    item = nltk.word_tokenize(out[i]["Tweet"])
    real_tokens.append(item)

hi


KeyError: 0

In [ ]:
#function generating feature dictionaries for tweets

def create_word_features(words, category):
    return dict([(word, category) for word in words])

#generated lists, full of dictionaries created by the create_word_features function
bot_tweets = [] + create_word_features(fake_tokens, "bot")
human_tweets = [] + create_word_features(real_tokens, "human")

In [ ]:
#ending index for training data
train_cutoff = 100001

#beginning index for testing data
test_begin = 100001

#beginning of index for human_tweets
human_begin = 10

train_set = bot_tweets[:100001]

#We increment by 10000 for the bot tweets because each testing set of data is expected to have approximately 1/10 of remaining training data.
#We add 1800 human-confirmed tweets to test accuracy for false positives

test_set = []

#Here we make a loop to create each individual dictionary (for testing)
for i in range(1, 11, 1): 
    test_set[i] = bot_tweets[test_begin:(test_begin) + 10000*i] + human_tweets[human_begin: (human_begin) + 1800]



In [ ]:
#This checks to see if the file has been run once

#pickle will help save the classifier state after the classifier has been trained
#pickling is important, as it allows python objecto hierarchies to turn into bitstreams

#If the file has not been run, run the code below
if run_var = 0:
    classifier = NaiveBayesClassifier.train(train_set)
    classify_buffer = open('russian_bots_positive_bayes.pickle', 'wb')
    pickle.dump(classifier, classify_buffer)
    classify_buffer.close()
    run_var += 1
    

#The alternative is re-opneing the classifier_buffer and updating the classifier    
else:
    classify_buffer = open('russian_bots_positive_bayes.pickle', 'rb')
    classifier = pickle.load(classify_buffer)
    classify_buffer.close()
    run_var += 1


In [ ]:
#We can use the simple testing suite from nltk to determine how accurate a classifier is
def accuracy_tester(classifier, test_set, test_set_index):
    print("Accuracy of test_set" + test_set_index + " is :", nltk.classify.util.accuracy(classifier, test_set) * 100)

#Here we print the accuracy of each of the test sets after running the trained classifier
for i in range(1, 11, 1):
    accuracy_tester(classifier, test_set, i);